In [1]:
import torch
from scripts import get_surface, get_sdf, scan
#from scripts.models import Segmentation_Network_full as Segmentation_Network
from scripts.utils import scale_to_unit_sphere, load_intra
from scripts.sampling_cube import CubeSampler
from scripts.dataset import CustomDataset
import numpy as np
from tqdm import tqdm

from collections import namedtuple 
import os
device='cuda:0'

# ACDC

In [3]:
dir_in = './ACDC_simplified'#'./IntrA3D/obj'
names = os.listdir(dir_in)
split_idx = len(names)//2

train_names = names[:split_idx]
test_names = names[split_idx:]

dirrectory = 'prepared_data/raw'
os.mkdir(dirrectory)

FileExistsError: [Errno 17] File exists: 'prepared_data/raw'

In [8]:
num_classes = 3

for name in tqdm(names):
    if os.path.exists(f'./{dirrectory}/{name[:-4]}'):
        continue
    
    
    train_dataset = CustomDataset(name,
                                  device='cpu',
                                  count=10,
                                  rotate_angle=1,
                                  dataset = 'acdc',
                                  num_classes=num_classes)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, num_workers=16, shuffle=True)
        
    train_data = []
    train_labels = []
    count=0
    for X, Y in (train_loader):
        X = X.reshape([-1,1, 16,16,16])
        Y = Y.reshape([-1, num_classes])

        train_data.append(X)
        train_labels.append(Y)
        
        count+=1
        if count==40:
            break
        
    new_train_data = torch.cat(train_data, dim=0)
    new_train_labels = torch.cat(train_labels, dim=0)

    os.mkdir(f'./{dirrectory}/{name[:-4]}')

    torch.save(new_train_data, f'./{dirrectory}/{name[:-4]}/data.pt')
    torch.save(new_train_labels, f'./{dirrectory}/{name[:-4]}/label.pt')

       

100%|█████████████████████████████████████████| 100/100 [54:37<00:00, 32.78s/it]


In [13]:
a = torch.load('./prepared_data/raw/patient001/data.pt')
a.shape

torch.Size([4000, 1, 16, 16, 16])

In [2]:
datadir = './prepared_data/raw/'
names = sorted(os.listdir(datadir))


groups = []
for i in range(1,100, 20):
    groups.append(np.arange(i, i+20))

np.random.seed(42)

test_names = []
for i in groups:
    test_names.append(np.random.choice(i, 10, replace=False))
test_names = list(np.concatenate(test_names))

for i in range(len(test_names)):
    name = test_names[i]
    if len(str(name)) == 1:
        test_names[i] = f'patient00{name}'
    elif len(str(name)) == 2:
        test_names[i] = f'patient0{name}'
    else:
        test_names[i] = f'patient{name}'
print('Models for test dataset: ', sorted(test_names))

Models for test dataset:  ['patient001', 'patient002', 'patient004', 'patient006', 'patient009', 'patient012', 'patient016', 'patient017', 'patient018', 'patient019', 'patient024', 'patient025', 'patient026', 'patient027', 'patient028', 'patient033', 'patient035', 'patient036', 'patient037', 'patient040', 'patient041', 'patient043', 'patient045', 'patient046', 'patient048', 'patient051', 'patient053', 'patient055', 'patient057', 'patient059', 'patient061', 'patient065', 'patient067', 'patient069', 'patient070', 'patient071', 'patient072', 'patient073', 'patient078', 'patient079', 'patient081', 'patient082', 'patient083', 'patient085', 'patient086', 'patient090', 'patient091', 'patient093', 'patient097', 'patient100']


In [3]:
train_data = []
train_labels = []

test_data = []
test_labels = []

for name in tqdm(names):
    
    data = torch.load(datadir+name+'/data.pt')
    labels = torch.load(datadir+name+'/label.pt')

    # equalizing the data
    # takes N of indicies of each class equal to least populated group which is base 
    epi = (labels[:,0] == 1).nonzero().squeeze()
    lv = (labels[:,1] == 1).nonzero().squeeze()
    rv = (labels[:,2] == 1).nonzero().squeeze()

    min_ = np.min([len(lv), len(rv), len(epi)])
 
    epi = np.random.choice(epi, min_, replace=False)
    lv = np.random.choice(lv, min_, replace=False)
    rv = np.random.choice(rv, min_, replace=False)

    if name in test_names:
        test_data.append(data[lv])
        test_labels.append(labels[lv])
    
        test_data.append(data[epi])
        test_labels.append(labels[epi])
        
        test_data.append(data[rv])
        test_labels.append(labels[rv])

    else: 
        train_data.append(data[lv])
        train_labels.append(labels[lv])
    
        train_data.append(data[epi])
        train_labels.append(labels[epi])
        
        train_data.append(data[rv])
        train_labels.append(labels[rv])
        

X_train = torch.concat(train_data, dim=0)
X_test = torch.concat(test_data, dim=0)

y_train = torch.concat(train_labels, dim=0)
y_test = torch.concat(test_labels, dim=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

100%|█████████████████████████████████████████| 100/100 [00:08<00:00, 11.92it/s]


(torch.Size([144918, 1, 16, 16, 16]),
 torch.Size([144867, 1, 16, 16, 16]),
 torch.Size([144918, 3]),
 torch.Size([144867, 3]))

In [4]:
dirr = './prepared_data/'
os.makedirs(os.path.join(dirr, 'train'))
os.makedirs(os.path.join(dirr, 'test'))

FileExistsError: [Errno 17] File exists: './prepared_data/train'

In [5]:
X_train = X_train.numpy()
y_train = y_train.numpy()
X_test = X_test.numpy()
y_test = y_test.numpy()

In [6]:
import h5py
with h5py.File(os.path.join(dirr, 'train', 'data.hdf5'), 'w') as f:
    f.create_dataset('points', data=X_train)
    f.create_dataset('labels', data=y_train)


with h5py.File(os.path.join(dirr, 'test', 'data.hdf5'), 'w') as f:
    f.create_dataset('points', data=X_test)
    f.create_dataset('labels', data=y_test)


In [7]:
from scripts.dataset import H5Dataset
from torch.utils.data import TensorDataset, DataLoader

In [8]:
train_dataset = H5Dataset('./prepared_data/train/data.hdf5')
test_dataset = H5Dataset('./prepared_data/test/data.hdf5')

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False)

len(train_dataset), len(test_dataset)

(144918, 144867)

In [12]:
next(iter(train_dataloader))[0].shape

torch.Size([256, 1, 16, 16, 16])

# Clinical Data

In [2]:
dir_in = './data_for_transfer_learning/obj'#'./IntrA3D/obj'
names = os.listdir(dir_in)
split_idx = len(names)//2

train_names = names[:split_idx]
test_names = names[split_idx:]

dirrectory = 'TL_prepared_data/raw'
#os.mkdir(dirrectory)

In [7]:
num_classes = 4

for name in tqdm(names):
    if os.path.exists(f'./{dirrectory}/{name[:-4]}'):
        continue
    
    
    train_dataset = CustomDataset(name,
                                  device='cpu',
                                  count=10,
                                  rotate_angle=1,
                                  dataset = 'clinical',
                                  num_classes=num_classes,
                                  data_dir=dir_in)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, num_workers=16, shuffle=True)
        
    train_data = []
    train_labels = []
    count=0
    for X, Y in (train_loader):
        X = X.reshape([-1,1, 16,16,16])
        Y = Y.reshape([-1, num_classes])

        train_data.append(X)
        train_labels.append(Y)
        
        count+=1
        if count==80:
            break
        
    new_train_data = torch.cat(train_data, dim=0)
    new_train_labels = torch.cat(train_labels, dim=0)

    os.mkdir(f'./{dirrectory}/{name[:-4]}')

    torch.save(new_train_data, f'./{dirrectory}/{name[:-4]}/data.pt')
    torch.save(new_train_labels, f'./{dirrectory}/{name[:-4]}/label.pt')

       

100%|███████████████████████████████████████████| 37/37 [29:33<00:00, 47.94s/it]


In [4]:
ls TL_prepared_data/raw/024

data.pt  label.pt


In [5]:
data = torch.load('./TL_prepared_data/raw/019/data.pt')
labels = torch.load('./TL_prepared_data/raw/019/label.pt')

In [6]:
sum(labels[:,0] == 1), sum(labels[:,1] == 1), sum(labels[:,2] == 1), sum(labels[:,3] == 1)

(tensor(1868), tensor(4208), tensor(1162), tensor(762))

In [4]:
datadir = './TL_prepared_data/raw/'
names = sorted(os.listdir(datadir))

np.random.seed(0)
test_names = np.random.choice(names, len(names)//2, replace=False)
print('Models for test dataset: ', sorted(test_names))

Models for test dataset:  ['003', '009', '011', '012', '014', '016', '017', '021', '023', '026', '029', '030', '031', '032', '033', '034', '035', '037']


In [5]:
train_data = []
train_labels = []

test_data = []
test_labels = []

for name in tqdm(names):
    
    data = torch.load(datadir+name+'/data.pt')
    labels = torch.load(datadir+name+'/label.pt')

    # equalizing the data
    # takes N of indicies of each class equal to least populated group which is base 
    lv = (labels[:,0] == 1).nonzero().squeeze()
    epi = (labels[:,1] == 1).nonzero().squeeze()
    rv = (labels[:,2] == 1).nonzero().squeeze()
    base = (labels[:,3] == 1).nonzero().squeeze()

    lv = np.random.choice(lv, len(base), replace=False)
    epi = np.random.choice(epi, len(base), replace=False)
    rv = np.random.choice(rv, len(base), replace=False)


    if name in test_names:
        test_data.append(data[lv])
        test_labels.append(labels[lv])
    
        test_data.append(data[epi])
        test_labels.append(labels[epi])
        
        test_data.append(data[rv])
        test_labels.append(labels[rv])
        
        test_data.append(data[base])
        test_labels.append(labels[base])

    else: 
        train_data.append(data[lv])
        train_labels.append(labels[lv])
    
        train_data.append(data[epi])
        train_labels.append(labels[epi])
        
        train_data.append(data[rv])
        train_labels.append(labels[rv])
        
        train_data.append(data[base])
        train_labels.append(labels[base])

X_train = torch.concat(train_data, dim=0)
X_test = torch.concat(test_data, dim=0)

y_train = torch.concat(train_labels, dim=0)
y_test = torch.concat(test_labels, dim=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

100%|███████████████████████████████████████████| 37/37 [00:05<00:00,  6.80it/s]


(torch.Size([46028, 1, 16, 16, 16]),
 torch.Size([43632, 1, 16, 16, 16]),
 torch.Size([46028, 4]),
 torch.Size([43632, 4]))

In [6]:
dirr = './TL_prepared_data/'
os.makedirs(os.path.join(dirr, 'train'))
os.makedirs(os.path.join(dirr, 'test'))

FileExistsError: [Errno 17] File exists: './TL_prepared_data/train'

In [7]:
X_train = X_train.numpy()
y_train = y_train.numpy()
X_test = X_test.numpy()
y_test = y_test.numpy()

In [8]:
X_train.dtype

dtype('float64')

In [12]:
import h5py

with h5py.File(os.path.join(dirr, 'train', 'data.hdf5'), 'w') as f:
    f.create_dataset('points', data=X_train)
    f.create_dataset('labels', data=y_train)


with h5py.File(os.path.join(dirr, 'test', 'data.hdf5'), 'w') as f:
    f.create_dataset('points', data=X_test)
    f.create_dataset('labels', data=y_test)


In [ ]:
from scripts.dataset import H5Dataset
from torch.utils.data import TensorDataset, DataLoader

In [12]:
train_dataset = H5Dataset('./TL_prepared_data/train/data.hdf5')

In [13]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

In [14]:
batch = next(iter(train_dataloader))